<a href="https://colab.research.google.com/github/yachtsman/Coursera_Capstone/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Segmenting and Clustering Neighborhoods in Toronto

Importing basic dependencies

In [1]:
import pandas as pd
import numpy as np
print('basic dependencies imported')

basic dependencies imported


Scraping data from wikipedia

In [2]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import csv

print('BeautifulSoup & CSV imported.')

BeautifulSoup & CSV imported.


In [5]:
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki_url,'lxml')
print('soup cooked')

soup cooked


In [6]:
wiki_table = soup.find('table',{'class':'wikitable sortable'})
wiki_rows = wiki_table.find_all('tr')

In [29]:
data = []
for row in wiki_rows:
    data.append([t.text.strip()
    for t in row.find_all('td')]
    )
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.drop([0], inplace=True) # to skip first rows 'none' 

Cleaning  the 'Not assigned' rows

In [30]:
df1 = df[df['Borough'] != 'Not assigned'].reset_index(drop=True) # to filter out 'Not assigned' rows
print(df1.head())
print(df1.shape)

  PostalCode           Borough                                Neighbourhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
(103, 3)


combinig the Neighborhoods with the equal postal codes 

In [31]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
print(df2.head())
print(df2.shape)

                Borough                           Neighbourhood
PostalCode                                                     
M1B         Scarborough                          Malvern, Rouge
M1C         Scarborough  Rouge Hill, Port Union, Highland Creek
M1E         Scarborough       Guildwood, Morningside, West Hill
M1G         Scarborough                                  Woburn
M1H         Scarborough                               Cedarbrae
(103, 2)


In [32]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df3 = df2.reset_index()

In [37]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
print('The final cleaned table looks like:')
print(df3.head(20))
print('with the shape:')
print(df3.shape)

The final cleaned table looks like:
   PostalCode      Borough                                      Neighbourhood
0         M1B  Scarborough                                     Malvern, Rouge
1         M1C  Scarborough             Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough                  Guildwood, Morningside, West Hill
3         M1G  Scarborough                                             Woburn
4         M1H  Scarborough                                          Cedarbrae
5         M1J  Scarborough                                Scarborough Village
6         M1K  Scarborough        Kennedy Park, Ionview, East Birchmount Park
7         M1L  Scarborough                    Golden Mile, Clairlea, Oakridge
8         M1M  Scarborough    Cliffside, Cliffcrest, Scarborough Village West
9         M1N  Scarborough                        Birch Cliff, Cliffside West
10        M1P  Scarborough  Dorset Park, Wexford Heights, Scarborough Town...
11        M1R  Scarborough  